In [1]:
# If we aren't on a c3 container, we need to get the c3 object.
try:
    c3
except:
    from c3python import get_c3
    #c3 = get_c3('http://0.0.0.0:8080', 'test', 'dtitraining_HC_v1', 'thick')
    c3 = get_c3('http://0.0.0.0:8080', 'test', 'Yifang_HC', 'thick')

Username: BA
Password: ········


In [2]:
import pandas as pd

raw_df = c3.BlockInfo.fetch(spec={
    'limit': -1,
    'filter': 'exists(prp_bf_lr)',
    'order': 'descending(id)',
    'include': 'pct_i_l,pct_t_l,prp_res_lr,pop10_ha_lr,hu10_ha_lr,eroom_ha_lr,med10_age,prp_bf_lr'
})

df = pd.DataFrame(raw_df.objs.toJson())
df.head()

,type,id,meta,version,med10_age,pct_i_l,pct_t_l,prp_res_lr,pop10_ha_lr,hu10_ha_lr,eroom_ha_lr,prp_bf_lr
0,BlockInfo,530359401004,"{'type': 'Meta', 'fetchInclude': '[pct_i_l,pct...",1,1978.0,15.504003,44.014487,0.771157,8.804554,4.048071,21.454775,0.075343
1,BlockInfo,530359401002,"{'type': 'Meta', 'fetchInclude': '[pct_i_l,pct...",1,1979.0,7.094398,59.561575,0.811293,1.808341,0.862481,5.174888,0.027872
2,BlockInfo,530359400002,"{'type': 'Meta', 'fetchInclude': '[pct_i_l,pct...",1,1985.0,2.941183,69.144427,0.917720,1.584815,0.775005,4.340028,0.017071
3,BlockInfo,530359400001,"{'type': 'Meta', 'fetchInclude': '[pct_i_l,pct...",1,1988.0,7.860382,51.629077,0.791335,3.992228,1.830507,10.250839,0.035959
4,BlockInfo,530350929022,"{'type': 'Meta', 'fetchInclude': '[pct_i_l,pct...",1,1979.0,11.596608,49.368363,0.981577,4.720380,1.686727,8.770982,0.040964


In [3]:
df.drop('meta', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)
df.drop('version', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.head()

,med10_age,pct_i_l,pct_t_l,prp_res_lr,pop10_ha_lr,hu10_ha_lr,eroom_ha_lr,prp_bf_lr
0,1978.0,15.504003,44.014487,0.771157,8.804554,4.048071,21.454775,0.075343
1,1979.0,7.094398,59.561575,0.811293,1.808341,0.862481,5.174888,0.027872
2,1985.0,2.941183,69.144427,0.917720,1.584815,0.775005,4.340028,0.017071
3,1988.0,7.860382,51.629077,0.791335,3.992228,1.830507,10.250839,0.035959
4,1979.0,11.596608,49.368363,0.981577,4.720380,1.686727,8.770982,0.040964


In [4]:
len(df)

19328

In [5]:
## checking the each column type ##
df.eroom_ha_lr.dtype

dtype('float64')

In [6]:
## filling all NAs with the mean value of that field.
df = df.fillna(df.mean())

In [7]:
##df.values

In [8]:
import numpy as np
import tensorflow as tf

In [9]:
np.random.seed(1014)

In [10]:
## random shaffling the dataset ##
values = df.values
np.random.shuffle(values)

## assign randomly 1:5
test_size = len(df) // 5    # test size floor (1/5) 
training, test = values[:-test_size], values[-test_size:]  # split by row to 

len(training), len(test)

(15463, 3865)

In [11]:
test

array([[1.97500000e+03, 2.99832402e+01, 3.31036623e+00, ...,
        4.06115954e+00, 2.80220008e+01, 1.19859383e-01],
       [1.99500000e+03, 2.11847751e+01, 3.05377163e+01, ...,
        4.80605780e+00, 4.32545202e+01, 1.05583448e-01],
       [1.93900000e+03, 2.38866856e+01, 3.69660057e+01, ...,
        9.07093534e+00, 8.16384181e+01, 1.71158778e-01],
       ...,
       [1.95400000e+03, 5.66980392e+01, 4.42352941e+00, ...,
        1.68836806e+01, 1.02990451e+02, 2.69082160e-01],
       [1.95600000e+03, 2.02797732e+01, 3.35179584e+01, ...,
        7.17964890e+00, 4.88216125e+01, 9.54553063e-02],
       [1.98400000e+03, 6.68272358e+01, 2.09349593e+00, ...,
        1.63279133e+01, 8.00067751e+01, 3.27474964e-01]])

In [12]:
## assinging features and labels, labels = many features with different correlations
training_features, training_labels = training[:, :-1], training[:, -1]
test_features, test_labels = test[:, :-1], test[:, -1]
test_labels

array([0.11985938, 0.10558345, 0.17115878, ..., 0.26908216, 0.09545531,
       0.32747496])

In [13]:
## adding 2 helper functions for normalization ##
def zero_one(x):
  training_min = x.min(axis=0)
  scale = x.max(axis=0) - training_min
  return training_min, scale

def standard(x):
  mean = x.mean(axis=0)
  std = x.std(axis=0)
  return mean, std

## normalization ##
offset, scale = zero_one(training_features)

training_features = (training_features - offset) / scale
test_features = (test_features - offset) / scale

test_features

array([[0.54545455, 0.30675941, 0.04418664, ..., 0.00681223, 0.00303954,
        0.0063554 ],
       [0.84848485, 0.21551865, 0.40761622, ..., 0.00980986, 0.00359705,
        0.00981013],
       [0.        , 0.24353767, 0.49342077, ..., 0.01351327, 0.00678906,
        0.01851561],
       ...,
       [0.22727273, 0.58379392, 0.05904509, ..., 0.02169867, 0.01263643,
        0.02335826],
       [0.25757576, 0.20613371, 0.44739637, ..., 0.01228413, 0.00537354,
        0.01107275],
       [0.68181818, 0.68883447, 0.0279439 , ..., 0.02024874, 0.01222048,
        0.01814555]])

In [14]:
## starting the tensorflow ##
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

from keras import backend as K

Using TensorFlow backend.


In [15]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [16]:
#training_features = np.array(training_features, dtype=np.float)
#test_features = np.array(test_features, dtype=np.float)
training_features = training_features.astype("float32")
test_features = test_features.astype("float32")

In [17]:
## tensorflow time ##
## model with Regularization 
model = Sequential()
model.add(Dense(300, input_shape=(7,), activation=relu, kernel_regularizer=l2(.001)))
model.add(Dense(300, activation=relu, kernel_regularizer=l2(.001)))
model.add(Dense(300, activation=relu, kernel_regularizer=l2(.001)))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(), metrics=['mse', 'msle', 'mape']) #, root_mean_squared_error
model.summary()
#model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mse', 'mae'])
history = model.fit(training_features, training_labels, batch_size=1000, epochs=200)
score = model.evaluate(test_features, test_labels, batch_size=1000)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               2400      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 183,301
Trainable params: 183,301
Non-trainable params: 0
_________________________________________________________________
Train on 15463 samples
Epoch 1/200
15463/15463 [==============================] - 2s 119us/sample - loss: 0.5301 - mse: 0.0092 - msle: 0.0063 - mape: 47.4646
Epoch 2/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.3468 -

15463/15463 [==============================] - 0s 20us/sample - loss: 0.0034 - mse: 0.0024 - msle: 0.0016 - mape: 26.0258
Epoch 56/200
15463/15463 [==============================] - 0s 19us/sample - loss: 0.0034 - mse: 0.0024 - msle: 0.0015 - mape: 25.8161
Epoch 57/200
15463/15463 [==============================] - 0s 20us/sample - loss: 0.0033 - mse: 0.0024 - msle: 0.0015 - mape: 26.1324
Epoch 58/200
15463/15463 [==============================] - 0s 23us/sample - loss: 0.0033 - mse: 0.0024 - msle: 0.0015 - mape: 26.1607
Epoch 59/200
15463/15463 [==============================] - 0s 26us/sample - loss: 0.0033 - mse: 0.0024 - msle: 0.0015 - mape: 25.8102
Epoch 60/200
15463/15463 [==============================] - 0s 24us/sample - loss: 0.0033 - mse: 0.0023 - msle: 0.0015 - mape: 25.8883
Epoch 61/200
15463/15463 [==============================] - 0s 22us/sample - loss: 0.0033 - mse: 0.0023 - msle: 0.0015 - mape: 25.5754
Epoch 62/200
15463/15463 [==============================] - 0s 19us/

15463/15463 [==============================] - 0s 22us/sample - loss: 0.0032 - mse: 0.0024 - msle: 0.0015 - mape: 26.0621
Epoch 116/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0032 - mse: 0.0024 - msle: 0.0015 - mape: 26.3013
Epoch 117/200
15463/15463 [==============================] - 0s 20us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.2901
Epoch 118/200
15463/15463 [==============================] - 0s 20us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.4062
Epoch 119/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.4651
Epoch 120/200
15463/15463 [==============================] - 0s 20us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.5826
Epoch 121/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.5429
Epoch 122/200
15463/15463 [==============================] - 0

Epoch 175/200
15463/15463 [==============================] - 0s 23us/sample - loss: 0.0030 - mse: 0.0023 - msle: 0.0015 - mape: 25.5713
Epoch 176/200
15463/15463 [==============================] - 0s 26us/sample - loss: 0.0030 - mse: 0.0023 - msle: 0.0015 - mape: 25.6204
Epoch 177/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0030 - mse: 0.0023 - msle: 0.0015 - mape: 25.5351
Epoch 178/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0030 - mse: 0.0023 - msle: 0.0015 - mape: 25.8499
Epoch 179/200
15463/15463 [==============================] - 0s 22us/sample - loss: 0.0031 - mse: 0.0023 - msle: 0.0015 - mape: 25.7481
Epoch 180/200
15463/15463 [==============================] - 0s 21us/sample - loss: 0.0031 - mse: 0.0024 - msle: 0.0015 - mape: 26.2669
Epoch 181/200
15463/15463 [==============================] - 0s 25us/sample - loss: 0.0030 - mse: 0.0023 - msle: 0.0015 - mape: 25.6573
Epoch 182/200
15463/15463 [=====================

['loss', 'mse', 'msle', 'mape']
[0.003120954592747163, 0.0024154698, 0.0015580124, 24.912813]


In [18]:
np.__version__

'1.18.1'

In [20]:
print(model.metrics_names)
print(score)

['loss', 'mse', 'msle', 'mape']
[0.003120954592747163, 0.0024154698, 0.0015580124, 24.912813]
